In [9]:
import torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from models.experimental import attempt_load
from utils.dataloaders import LoadImagesAndLabels
from utils.general import non_max_suppression
from models.experimental import attempt_load
import cv2
# Specify the absolute paths to the YOLOv5s model files
model_weights_path = "/Users/devnath/yolov5/yolov5m.pt"
model_config_path = "/Users/devnath/yolov5/models/yolov5m.yaml"


In [10]:
# Define your custom dataset class
import glob
import os
from PIL import Image
import torch
from torch.utils.data import Dataset


class CustomDataset(Dataset):
    def __init__(self, data_dir, img_size=640):
        self.img_size = img_size
        self.dataset = LoadImagesAndLabels(data_dir, img_size=img_size)
        self.transforms = transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ])

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img, labels = self.dataset[idx]

        # Resize the image
        img = cv2.resize(img, (self.img_size, self.img_size))

        # Convert labels to YOLO format (center coordinates, width, height)
        # labels should be a list of bounding boxes in the format: [x, y, w, h, class]
        # Perform this conversion if your labels are not in YOLO format
        labels = convert_labels_to_yolo_format(labels, img.shape)

        # Normalize image pixel values and convert to torch tensor
        img = img / 255.0
        img = torch.from_numpy(img).permute(2, 0, 1).float()

        return img, labels

In [11]:
# Create train and validation datasets
train_dataset = CustomDataset('/Users/devnath/Desktop/Potholes_data/train')
val_dataset = CustomDataset('/Users/devnath/Desktop/Potholes_data/val')

Scanning /Users/devnath/Desktop/Potholes_data/train.cache... 144 images, 0 backgrounds, 0 corrupt: 100%|██████████| 144/144 [00:00<?, ?it/s]
train: WARNING ⚠️ /Users/devnath/Desktop/Potholes_data/train/142.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /Users/devnath/Desktop/Potholes_data/train/143.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /Users/devnath/Desktop/Potholes_data/train/144.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /Users/devnath/Desktop/Potholes_data/train/146.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /Users/devnath/Desktop/Potholes_data/train/147.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /Users/devnath/Desktop/Potholes_data/train/148.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /Users/devnath/Desktop/Potholes_data/train/149.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /Users/devnath/Desktop/Potholes_data/train/150.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /Users/devnath/Desktop/Po

In [12]:
# Create data loaders
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)


In [13]:
# Load YOLOv5 model
# model = attempt_load("yolov5s")  # Adjust model size as needed
# model = attempt_load(model_weights_path, model_config_path)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the model
model = attempt_load(model_weights_path).to(device)

# Define loss function (You might need a custom YOLOv5-specific loss function)
criterion = torch.nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Define the number of epochs
num_epochs = 10


Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients


In [14]:
# Training loop
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    total_loss = 0.0

    for batch_idx, (images, labels) in enumerate(train_loader):
        # Forward pass
        outputs = model(images)

        # Calculate loss
        loss = criterion(outputs, labels)
        total_loss += loss.item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_idx % 10 == 0:
            print(f"Epoch [{epoch + 1}/{num_epochs}], Batch [{batch_idx}/{len(train_loader)}], Loss: {loss.item()}")

    print(f"Epoch [{epoch + 1}/{num_epochs}], Average Loss: {total_loss / len(train_loader)}")

    
torch.save(model.state_dict(), 'yolov5_custom_model.pth')


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


Traceback (most recent call last):
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "<string>", line 1, in <module>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/devnath/opt/anaconda3/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
  File "/Users/devnath/opt/anaconda3/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
  File "/Users/devnath/opt/anaconda3/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/devnath/opt/anaconda3/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/devnath/opt/anaconda3/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/devnath/opt/anaconda3/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
Attribute

RuntimeError: DataLoader worker (pid(s) 3316) exited unexpectedly